In [2]:
from collections import defaultdict
from pymongo import MongoClient
from dotenv import load_dotenv
import pandas as pd
import json
import os
import re

In [3]:
mongo_host = '127.0.0.1'
mongo_port = '27017'
mongo_db = 'raw_data'
connection_string = f"mongodb://{mongo_host}:{mongo_port}/"

In [4]:
collection_name = 'okjob_test'

In [5]:
client = MongoClient(connection_string)
db = client[mongo_db]
collection = db[collection_name]

In [6]:
display(db['okjob_test'].find_one())

{'_id': ObjectId('661ebfe686c81b1febea47f5'),
 'Company-ID': '1810',
 'LinkedIn-Job-Link': 'https://www.linkedin.com/jobs/view/strategic-account-executive-at-signifyd-3762391942',
 'Company-Name': 'Signifyd',
 'Job-Title': 'Strategic Account Executive',
 'Location': 'USA',
 'Job-Description': 'data/processed/okjob.io/full_job_description/okjob_jobdesc_id=3762391942.html',
 'Apply-Link': 'https://www.linkedin.com/jobs/view/3762391942/',
 'Region': 'North America',
 'Job-Type': 'Remote,100% Salary, Four Days',
 'Job-Tags': 'eCommerce, SaaS, RiskManagement, SalesStrategy, BusinessAnalytics',
 'Job-Category': 'Sales & Account Management, Business Development',
 'Hours': '32',
 'Salary-Min': '160000',
 'Salary-Max': '185000',
 'sourceId': '3762391942'}

In [40]:
query = {
    "$and": [
        {"sourceId": {"$ne": ""}},
        {"Job-Title": {"$regex": "data", "$options": "i"}},  # Case-insensitive search for "data" in jobTitle
        {"Location": {"$ne": ""}},
        {"Salary-Min": {"$ne": ""}},
        {"Salary-Max": {"$ne": ""}},
        {"LinkedIn-Job-Link": {"$ne": ""}},
        {"Job-Type": {"$ne": ""}},
        {"Job-Tags": {"$ne": ""}}
      # {"currency": {"$ne": ""}}
    ]
}
projection = {
    "sourceId": 1,
    "Job-Title": 1,
    "Location": 1,
    "Salary-Min": 1,
    "Salary-Max": 1,
    "LinkedIn-Job-Link": 1,
    "Job-Type": 1,
    "Job-Tags": 1,
    "_id": 0
}

documents = collection.find(query, projection)
data_objects = []
for doc in documents:
    data_objects.append(doc)
df_ok = pd.DataFrame(data_objects)
df_ok['Salary-Min'] = pd.to_numeric(df_ok['Salary-Min'], errors='coerce')
df_ok['Salary-Max'] = pd.to_numeric(df_ok['Salary-Max'], errors='coerce')
# didn't dropna or filter salary-min, otherwise the dataset is very small, we should fill them with mean/mode later
df_ok.head(5)




,LinkedIn-Job-Link,Job-Title,Location,Job-Type,Job-Tags,sourceId,Salary-Min,Salary-Max
0,https://uk.linkedin.com/jobs/view/junior-data-...,Junior Data Engineer,"Framwellgate Moor, England, United Kingdom","Hybrid, Remote, Four Days","SQL, ETL, Python, Kafka, BigQuery",3790819042,NaN,NaN
1,https://uk.linkedin.com/jobs/view/catalogue-da...,Catalogue Data Analyst,"Framwellgate Moor, England, United Kingdom","Hybrid, Remote, Four Days","SQL, DataGovernance, DataWarehousing, Metadata...",3791431753,NaN,NaN
2,https://mx.linkedin.com/jobs/view/data-scienti...,Data Scientist - LATAM,"Mexico City, Mexico","Hybrid, 100% Salary, Four Days","Python, Java, SQL, Machine Learning, Data Anal...",3795251305,NaN,NaN
3,https://br.linkedin.com/jobs/view/data-scienti...,Data Scientist - LATAM,"São Paulo, São Paulo, Brazil","Hybrid, 100% Salary, Four Days","Python, Java, SQL, Machine Learning, Data Anal...",3795248714,NaN,NaN
4,https://www.linkedin.com/jobs/view/data-engine...,Data Engineer,United States,"Remote, 100% Salary, Four Days","SQL, Python, BigQuery, Dataflow, Airflow",3755031443,98000.0,98100.0


In [41]:
df_ok['Salary-Min'] = df_ok['Salary-Min'].fillna(df_ok['Salary-Min'].mean())
df_ok['Salary-Max'] = df_ok['Salary-Max'].fillna(df_ok['Salary-Max'].mean())


In [42]:
df_ok.head(5)

,LinkedIn-Job-Link,Job-Title,Location,Job-Type,Job-Tags,sourceId,Salary-Min,Salary-Max
0,https://uk.linkedin.com/jobs/view/junior-data-...,Junior Data Engineer,"Framwellgate Moor, England, United Kingdom","Hybrid, Remote, Four Days","SQL, ETL, Python, Kafka, BigQuery",3790819042,98000.0,98100.0
1,https://uk.linkedin.com/jobs/view/catalogue-da...,Catalogue Data Analyst,"Framwellgate Moor, England, United Kingdom","Hybrid, Remote, Four Days","SQL, DataGovernance, DataWarehousing, Metadata...",3791431753,98000.0,98100.0
2,https://mx.linkedin.com/jobs/view/data-scienti...,Data Scientist - LATAM,"Mexico City, Mexico","Hybrid, 100% Salary, Four Days","Python, Java, SQL, Machine Learning, Data Anal...",3795251305,98000.0,98100.0
3,https://br.linkedin.com/jobs/view/data-scienti...,Data Scientist - LATAM,"São Paulo, São Paulo, Brazil","Hybrid, 100% Salary, Four Days","Python, Java, SQL, Machine Learning, Data Anal...",3795248714,98000.0,98100.0
4,https://www.linkedin.com/jobs/view/data-engine...,Data Engineer,United States,"Remote, 100% Salary, Four Days","SQL, Python, BigQuery, Dataflow, Airflow",3755031443,98000.0,98100.0


In [43]:
# Categorize job titles
import spacy
nlp = spacy.load('en_core_web_sm')
def categorize_seniority(job_title):
    doc = nlp(job_title)
    # List of keywords (lemmas) to look for
    keywords_senior = ['strategic', 'principal', 'staff', 'lead', 'senior', 'head']
    keywords_junior = ['trainee', 'junior', 'apprentice', 'entry level' ]
    # Check if any token's lemma is in our keywords list
    if any(token.lemma_.lower() in keywords_senior for token in doc):
        return 'Senior'
    elif any(token.lemma_.lower() in keywords_junior for token in doc):
        return 'Junior'
    else:
        return 'Any'

# Apply the function to create a new column
df_ok['jobLevel'] = df_ok['Job-Title'].apply(categorize_seniority)


In [44]:
# Define your lists of skills, technologies, and site preferences

keywords_skills = [
    "SQL", "Structured Query Language", "Python", "R", "Docker", "AWS", "Amazon Web Services",
    "Azure", "Google Cloud Platform", "GCP", "Snowflake", "Hadoop", "Spark", "Kubernetes",
    "Jenkins", "BI", "Business Intelligence", "Tableau", "Power BI", "Looker", "ETL",
    "Extract Transform Load", "Informatica", "Talend", "SSIS", "CRM",
    "Customer Relationship Management", "Salesforce", "SAP", "Git", "NoSQL", "MongoDB",
    "Cassandra", "PostgreSQL", "MySQL", "Data Modeling", "Machine Learning", "ML", "AI",
    "Apache Kafka", "Redis", "Elasticsearch", "Kibana", "Ansible", "REST", "RESTful", "API",
    "GraphQL", "Linux", "Matplotlib", "Seaborn", "Jupyter Notebook", "Scikit-learn",
    "TensorFlow", "PyTorch", "Data Lakes", "Data Warehousing", "Agile", "Scrum", "Blockchain",
    "Edge Computing", "VMware", "SAS", "Flask", "Django", "Apache", "Airflow", "Luigi", "NLP",
    "Databricks", "redshift", "Excel", "HANA", "Oracle", "crypto"
]

keywords_site = ['remote', 'hybrid', 'on-site']
# Function to categorize job titles and descriptions by keywords, accepting a keyword list as a parameter
def categorize_by_keywords(text, keywords):
    doc = nlp(text)
    # Initialize an empty set to avoid duplicates
    keywords_found = set()
    # Check each token in the text
    for token in doc:
        # Normalize the token's text for case-insensitive matching
        token_text = token.text.lower()
        # If the normalized token is in our list of keywords, add the original token text to the set
        if token_text in [keyword.lower() for keyword in keywords]:
            keywords_found.add(token.text)
    # Return a comma-separated string of unique keywords found, or "None" if no keywords were identified
    return ', '.join(keywords_found) if keywords_found else "None"

# Applying the function to each row for both jobSkills and jobSite columns
df_ok['jobSkills'] = df_ok.apply(lambda row: categorize_by_keywords(row['Job-Type'] + " " + row['Job-Tags'], keywords_skills), axis=1)
df_ok['jobSite'] = df_ok.apply(lambda row: categorize_by_keywords(row['Job-Type'] + " " + row['Job-Tags'], keywords_site), axis=1)





In [45]:
# categorize titles
keywords_title = {
    "data administrator": ["data", "administrator", "entry", "protection", "officer", "clerk", "admin", "migration", "cleanser", "inputter", "coordinator", "assistant", "pocessor", "auditor", "governance", "apprentice", "executive", "manager"],
    "data engineer": ["data", "engineer", "developer", "engineering", "modeller", "technical"],
    "data analyst": ["data", "analyst", "analytics", "analysis", "investigation", "workstream", "visualisation", "insight", "consultant"],
    "database administrator": ["database", "administrator", "assistant", "manager"],
    "data scientist": ["data", "scientist", "science", "engineer"],
    "data center": ["data", "center", "cabling", "installer", "installation", "engineer"],
    "data test": ["data", "test", "tester", "automation", "processing"],
    "data architect": ["data", "architect"],
    "manager": ["head", "manager", "director", "procurement", "management"]  
}
def categorize_job_titles(job_title, keywords_title):
    # Prepare the job title: lowercase, remove special characters, and split into words
    words = re.sub('[^a-z0-9\s]', '', job_title.lower()).split()
    
    # Initialize a dictionary to hold the count of matches for each category
    matches = defaultdict(int)
    # Initialize a dictionary to hold the sum of indexes for matched words for tie-breaking
    index_sums = defaultdict(int)
    
    for category, keywords in keywords_title.items():
        for word in words:
            if word in keywords:
                matches[category] += 1
                # Sum the indexes of matched words for tie-breaking
                index_sums[category] += keywords.index(word)
    
    if not matches:
        return 'Other'
    
    # Find the category(ies) with the maximum count of matches
    max_matches = max(matches.values())
    candidates = [category for category, count in matches.items() if count == max_matches]
    
    # If there's a single best match, return it
    if len(candidates) == 1:
        return candidates[0]
    
    # If there are ties, use the sum of indexes for tie-breaking
    return min(candidates, key=lambda category: index_sums[category])
# Apply the function to the 'jobTitle' column and create a new 'jobCategory' column
df_ok['jobCategory'] = df_ok['Job-Title'].apply(lambda x: categorize_job_titles(x, keywords_title))



In [46]:
df_ok = df_ok[['jobCategory', 'jobLevel', 'jobSkills', 'jobSite', 'Salary-Min', 'Salary-Max', 'sourceId','Job-Title','LinkedIn-Job-Link','Location' ]]
df_ok.drop_duplicates(inplace=True)

In [47]:
def clean_sort_and_deduplicate(text):
    if not isinstance(text, str):
        return text
    # Lowercase, strip whitespace, and split on commas
    parts = text.lower().strip().split(',')
    # Remove duplicates and sort
    cleaned_parts = sorted(set(part.strip() for part in parts))
    # Join the cleaned parts back into a single string
    return ', '.join(cleaned_parts)


In [48]:
# Apply the modified function to sort the terms within the 'jobSite' column using .loc
df_ok.loc[:, 'jobSite'] = df_ok['jobSite'].apply(clean_sort_and_deduplicate)

In [49]:
# Mapping of source column names to postgres db column names
column_mappings = {
    'sourceId': 'source_id',
    'Job-Title': 'job_title_name',
    'jobLevel': 'experience_level',
    'Salary-Min': 'salary_min',
    'Salary-Max': 'salary_max',
    'LinkedIn-Job-Link': 'joboffer_url',
    'Location': 'location_country',
    'jobSite': 'job_site',
    'jobSkills': 'skills',
    'jobCategory': 'categories'
}
additional_columns = {
    'data_source_name': 'ok'
}
# Rename columns based on the mapping
df_ok.rename(columns=column_mappings, inplace=True)
df_ok.head()

,categories,experience_level,skills,job_site,salary_min,salary_max,source_id,job_title_name,joboffer_url,location_country
0,data engineer,Junior,"ETL, SQL, Python","hybrid, remote",98000.0,98100.0,3790819042,Junior Data Engineer,https://uk.linkedin.com/jobs/view/junior-data-...,"Framwellgate Moor, England, United Kingdom"
1,data analyst,Any,SQL,"hybrid, remote",98000.0,98100.0,3791431753,Catalogue Data Analyst,https://uk.linkedin.com/jobs/view/catalogue-da...,"Framwellgate Moor, England, United Kingdom"
2,data scientist,Any,"SQL, Python",hybrid,98000.0,98100.0,3795251305,Data Scientist - LATAM,https://mx.linkedin.com/jobs/view/data-scienti...,"Mexico City, Mexico"
3,data scientist,Any,"SQL, Python",hybrid,98000.0,98100.0,3795248714,Data Scientist - LATAM,https://br.linkedin.com/jobs/view/data-scienti...,"São Paulo, São Paulo, Brazil"
4,data engineer,Any,"SQL, Python, Airflow",remote,98000.0,98100.0,3755031443,Data Engineer,https://www.linkedin.com/jobs/view/data-engine...,United States


In [50]:
df_ok_postgres = df_ok[
    [column_mappings.get(col, col) for col in column_mappings.keys()]
]

In [51]:
# Add additional columns with default values
for col, default_value in additional_columns.items():
    df_ok_postgres[col] = default_value

/var/folders/9p/t42qr2k15v3b0fnfl840bhjh0000gn/T/ipykernel_41916/3282893265.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ok_postgres[col] = default_value


In [52]:
# Ensure salaries are integers and greater than 0
df_ok_postgres['salary_min'] = pd.to_numeric(df_ok_postgres['salary_min'], errors='coerce').fillna(0).astype(int)
df_ok_postgres['salary_max'] = pd.to_numeric(df_ok_postgres['salary_max'], errors='coerce').fillna(0).astype(int)
df_ok_postgres = df_ok_postgres[(df_ok_postgres['salary_min'] > 0) & (df_ok_postgres['salary_max'] > 0)]
# Set the types for other fields as strings
df_ok_postgres['source_id'] = df_ok_postgres['source_id'].astype(str)
df_ok_postgres['experience_level'] = df_ok_postgres['experience_level'].astype(str)
df_ok_postgres['joboffer_url'] = df_ok_postgres['joboffer_url'].astype(str)
df_ok_postgres['currency_symbol'] = df_ok_postgres['currency_symbol'].astype(str)
df_ok_postgres['location_country'] = df_ok_postgres['location_country'].astype(str)
df_ok_postgres['data_source_name'] = df_ok_postgres['data_source_name'].astype(str)
df_ok_postgres['skills'] = df_ok_postgres['skills'].astype(list)
df_ok_postgres['categories'] = df_ok_postgres['categories'].astype(list)
df_ok_postgres['job_site'] = df_ok_postgres['job_site'].astype(str)

/var/folders/9p/t42qr2k15v3b0fnfl840bhjh0000gn/T/ipykernel_41916/3841136033.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ok_postgres['salary_min'] = pd.to_numeric(df_ok_postgres['salary_min'], errors='coerce').fillna(0).astype(int)
/var/folders/9p/t42qr2k15v3b0fnfl840bhjh0000gn/T/ipykernel_41916/3841136033.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ok_postgres['salary_max'] = pd.to_numeric(df_ok_postgres['salary_max'], errors='coerce').fillna(0).astype(int)


In [ ]:
# placeholder for country column

In [ ]:
# placeholder for currency and publoished date